In [ ]:
from IPython.display import HTML

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import os
import numpy as np

In [ ]:
fname = os.path.join('..','results','rollingStats.mp4')

In [ ]:
# the input file:
filename = 'opendap_hyrax_large_format_RS03ASHS-PN03B-06-CAMHDA301_2016_01_01_CAMHDA301-20160101T210000Z.mp4'

In [ ]:
# the results directory
results_path = os.path.join(os.getcwd(),'..','results')

In [ ]:
# the directory of the input file
video_path = os.path.join(os.getcwd(),'..','OOIVideos')

In [ ]:
video_encoded = open(fname, "rb").read().encode("base64")
video_tag = '<video width="800" height="800" controls alt="test" src="data:video/{0};base64,{1}">'.format("mp4", video_encoded)
HTML(data=video_tag)

#### Comparing rolling statistics of two sequences (two consecutive days).

In [ ]:
# rolling_stats are pickled in store.pckl and store.pckl1

In [ ]:
f= open(os.path.join('..','results','store.pckl'),'rb')

In [ ]:
f1 = open(os.path.join('..','results','store.pckl1'),'rb')

In [ ]:
import pickle

In [ ]:
res = pickle.load(f)
res1 = pickle.load(f1)

In [ ]:
f.close()
f1.close()

In [ ]:
plt.figure(figsize = (15,5))
plt.plot(res[0])
plt.plot(res1[0],'r')
plt.title('Rolling Average')

Seems scenes start to get offset toward the end of the video.

In [ ]:
plt.figure(figsize = (15,5))
plt.plot(res[1])
plt.plot(res1[1],'r')
plt.title('Rolling Variance')

Seems frames start to get offse

##### Scene Extraction

We will try to extract scenes based on the rolling variance: we will look for areas where the variance is relatively small.

In [ ]:
plt.figure(figsize = (15,5))
plt.plot(res[1])
plt.title('Rolling Variance')

In [ ]:
# using 2*median as a threshold: the assumption is that the cutoff threshold is well above the median
thresh = 2*np.median(res[1])
binary = np.array(np.array(res[1])<thresh)

In [ ]:
np.median(res[1])

In [ ]:
# of time steps in scenes
sum(binary)

In [ ]:
# removing chunks
plt.figure(figsize = (15,5))
plt.plot(np.array(res[1]))
plt.plot(np.array(res[1])*((np.array(res[1])<thresh)),'r')
plt.title('Cut-off Peaks')

In [ ]:
def extractSceneBounds(rolling_var):
    """
        This function extracts the bounds of the scenes.
        
        
        Input: rolling_var
        Output: list of tuples: starting and ending time point for each scene
        
    """
    
    thresh = 2*np.median(rolling_var)
    binary = np.array(np.array(res[1])<thresh).astype('int')
    diff = binary[1:] - binary[:-1]
    upper_bound = np.where(diff == 1)[0]
    lower_bound = np.where(diff == -1)[0]
    
    return(zip(list(upper_bound),list(lower_bound)))
    
    

In [ ]:
bounds = extractSceneBounds(res[1])

In [ ]:
print(bounds)

In [ ]:
# white regions are scenes
plt.figure(figsize = (10,3))
plt.imshow(np.reshape(binary,(1,len(res[1]))),aspect = 'auto',cmap = 'gray')

In [ ]:
plt.figure(figsize = (15,5))
plt.plot(res[1])
for x in np.array(bounds).ravel():
    plt.axvline(x,color = 'r')

In [ ]:
import imageio

In [ ]:
# this function is really slow
def createSlicedVideo(binary, moviename,videoname, subsampleRate, speedup):
    
    
    vid_out = imageio.get_writer(videoname,fps = 30/subsampleRate*speedup)
    vid_in = imageio.get_reader(moviename,'ffmpeg')
    
    frame_idx = np.arange(0,len(vid_in),subsampleRate)
    
    fig = plt.figure()

    for i in range(len(binary)):
    #for i in range(60):
        
        print(i)
    
        
        im = vid_in.get_data(frame_idx[i])
        plt.subplot(211)
        plt.imshow(im*binary[i], aspect = 'equal')
        plt.axis('off')

    
        plt.subplot(212)
        plt.imshow(im*(1-binary[i]), aspect = 'equal')
        plt.axis('off')
    
    
        # convert the plot
        fig.canvas.draw()
        fig_data = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
        fig_data = fig_data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        
        vid_out.append_data(fig_data.astype(np.dtype('uint8')))
    
    vid_in.close()
    vid_out.close()

In [ ]:
# this version is faster
def createSlicedVideo(binary, moviename,videoname, subsampleRate, speedup):
    """
        createSlicedVideo(binary, moviename,videoname, subsampleRate, speedup)
        
        It creates a video which masks the motion with black frames
        
        Input: 
        -----
        
        binary: a binary array of length number of subsampled frames (1 - scene, 0 - motion)
        
        
    """
    
    
    vid_out = imageio.get_writer(videoname,fps = 30/subsampleRate*speedup)
    vid_in = imageio.get_reader(moviename,'ffmpeg')
    
    frame_idx = np.arange(0,len(vid_in),subsampleRate)
    
    fig = plt.figure()

    for i in range(len(binary)):
    #for i in range(60):
        
        print(i)
    
        
        im = vid_in.get_data(frame_idx[i])
        vid_out.append_data(im*binary[i])
    
    vid_in.close()
    vid_out.close()

In [ ]:
# unselect to save video
# createSlicedVideo(binary,os.path.join(video_path,filename),os.path.join(results_path,'slicedVideo.mp4'),10,10)

In [ ]:
# unselect to save video
# createSlicedVideo(1- binary,os.path.join(video_path,filename),os.path.join(results_path,'slicedVideo_motion.mp4'),10,10)

In [ ]:
video_encoded = open(os.path.join(results_path,'slicedVideo_static.mp4'), "rb").read().encode("base64")
video_tag = '<video width="320" height="320" controls alt="test" src="data:video/{0};base64,{1}">'.format("mp4", video_encoded)
HTML(data=video_tag)

In [ ]:
video_encoded = open(os.path.join(results_path,'slicedVideo_motion.mp4'), "rb").read().encode("base64")
video_tag = '<video width="320" height="320" controls alt="test" src="data:video/{0};base64,{1}">'.format("mp4", video_encoded)
HTML(data=video_tag)